# Join Statements - Lab

## Introduction

In this lab, you'll practice your knowledge of `JOIN` statements, using various types of joins and various methods for specifying the links between them.

## Objectives

You will be able to:
* Write SQL queries that make use of various types of joins
* Compare and contrast the various types of joins
* Discuss how primary and foreign keys are used in SQL
* Decide and perform whichever type of join is best for retrieving desired data

## CRM Schema

In almost all cases, rather than just working with a single table you will typically need data from multiple tables. 
Doing this requires the use of **joins** using shared columns from the two tables. 

In this lab, you'll use the same customer relationship management (CRM) database that you saw from the previous lesson.
<img src='images/Database-Schema.png' width="600">

## Connecting to the Database
Import the necessary packages and connect to the database `'data.sqlite'`.

In [1]:
# Your code here
import pandas as pd
import sqlite3
conn = sqlite3.connect('data.sqlite')
cur = conn.cursor()


## Display the names of all the employees in Boston 

Hint: join the employees and offices tables.

In [3]:
# Your code here
cur.execute('''SELECT e.*, o.city 
               FROM employees e 
               JOIN offices o 
               USING(officeCode)
               WHERE city == 'Boston'; ''')
df = pd.DataFrame(cur.fetchall())
df.columns = list(i[0] for i in cur.description)
df

,employeeNumber,lastName,firstName,extension,email,officeCode,reportsTo,jobTitle,city
0,1188,Firrelli,Julie,x2173,jfirrelli@classicmodelcars.com,2,1143,Sales Rep,Boston
1,1216,Patterson,Steve,x4334,spatterson@classicmodelcars.com,2,1143,Sales Rep,Boston


## Are there any offices that have zero employees?
Hint: Combine the employees and offices tables and use a group by.

In [9]:
# Your code here
cur.execute('''SELECT o.officeCode, o.city, COUNT(e.employeeNumber) AS n_employees
               FROM offices o
               LEFT JOIN employees e
               USING(officeCode)
               GROUP BY officeCode
               HAVING n_employees = 0;''')
df2 = pd.DataFrame(cur.fetchall())
df2.columns = [i[0] for i in cur.description]
df2

,officeCode,city,n_employees
0,27,Boston,0


## Write 3 Questions of your own and answer them

In [ ]:
# Answers will vary
# Example: Display the htmlDescription and employee's first and last name for each product that each employee has sold

In [28]:
# Your code here
#Which Sales Rep Sold to the most businesses?
cur.execute('''SELECT e.employeeNumber SalesRep, e.firstName, e.lastName, COUNT(c.customerNumber) Num_of_Businesses
               FROM employees e
               JOIN customers c
               ON e.employeeNumber = c.salesRepEmployeeNumber
               GROUP BY SalesRep
               ORDER BY Num_of_Businesses DESC;''')
df3 = pd.DataFrame(cur.fetchall())
df3.columns = [i[0] for i in cur.description]
df3

,SalesRep,firstName,lastName,Num_of_Businesses
0,1401,Pamela,Castillo,10
1,1504,Barry,Jones,9
2,1501,Larry,Bott,8
3,1323,George,Vanauf,8
4,1370,Gerard,Hernandez,7
5,1286,Foon Yue,Tseng,7
6,1702,Martin,Gerard,6
7,1337,Loui,Bondur,6
8,1216,Steve,Patterson,6
9,1188,Julie,Firrelli,6


In [47]:
# Your code here
#What were the amounts invidiaul sales reps accumulated?
cur.execute('''SELECT c.salesRepEmployeeNumber SalesRep, SUM(p.amount) Total_Amt_Sold
               FROM customers c
               JOIN payments p
               USING(customerNumber)
               GROUP BY SalesRep
               ORDER BY Total_Amt_Sold Desc;''')
df4 = pd.DataFrame(cur.fetchall())
df4.columns = [i[0] for i in cur.description]
df4

,SalesRep,Total_Amt_Sold
0,1370,1112003.81
1,1165,989906.55
2,1401,750201.87
3,1501,686653.25
4,1504,637672.65
5,1323,584406.80
6,1337,569485.75
7,1611,509385.82
8,1612,497907.16
9,1286,488212.67


In [48]:
# Your code here
#which sales rep sold the most?
df3.set_index('SalesRep').join(df4.set_index('SalesRep')).sort_values(by='Total_Amt_Sold', ascending=False)

,firstName,lastName,Num_of_Businesses,Total_Amt_Sold
SalesRep,,,,
1370,Gerard,Hernandez,7,1112003.81
1165,Leslie,Jennings,6,989906.55
1401,Pamela,Castillo,10,750201.87
1501,Larry,Bott,8,686653.25
1504,Barry,Jones,9,637672.65
1323,George,Vanauf,8,584406.80
1337,Loui,Bondur,6,569485.75
1611,Andy,Fixter,5,509385.82
1612,Peter,Marsh,5,497907.16


## Level Up: Display the names of every individual product that each employee has sold

In [49]:
# Your code here
cur.execute("""SELECT firstName, lastName, productName
               FROM employees e
               JOIN customers c
               ON e.employeeNumber = c.salesRepEmployeeNumber
               JOIN orders o
               USING(customerNumber)
               JOIN orderdetails od
               USING(orderNumber)
               JOIN products p
               USING(productCode)""")
df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
print(len(df))
df.head()

## NOTE: This question could also be answered using a HAVING clause which you learned about previously 
## Remember, HAVING clauses are filters similar to the WHERE clause but are conditions applied after a GROUP BY.

2996


,firstName,lastName,productName
0,Leslie,Jennings,1958 Setra Bus
1,Leslie,Jennings,1940 Ford Pickup Truck
2,Leslie,Jennings,1939 Cadillac Limousine
3,Leslie,Jennings,1996 Peterbilt 379 Stake Bed with Outrigger
4,Leslie,Jennings,1968 Ford Mustang


## Level Up: Display the Number of Products each employee has sold

In [52]:
# Your code here
cur.execute("""SELECT firstName, lastName, COUNT(productName)
               FROM employees e
               JOIN customers c                   
               ON e.employeeNumber = c.salesRepEmployeeNumber
               JOIN orders o 
               USING(customerNumber)
               JOIN orderdetails od 
               USING(orderNumber)                    
               JOIN products p 
               USING(productCode)
               GROUP BY lastName
               ORDER BY firstName""")
df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
print(len(df))
df

## NOTE: Another way to access this information would be to use the dataframe from 
## the previous "level up" question and call a pandas .groupby method like so:
# df.groupby(['firstName','lastName']).count()

15


,firstName,lastName,COUNT(productName)
0,Andy,Fixter,185
1,Barry,Jones,220
2,Foon Yue,Tseng,142
3,George,Vanauf,211
4,Gerard,Hernandez,396
5,Julie,Firrelli,124
6,Larry,Bott,236
7,Leslie,Jennings,331
8,Leslie,Thompson,114
9,Loui,Bondur,177


## Summary

Congrats! You practiced using join statements and leveraged your foreign keys knowledge!